In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

#Define the Ticker
ticker_symbol = "SPY"
ticker = yf.Ticker(ticker_symbol)

# Get all available expiration dates
expirations = ticker.options
print(f"Found {len(expirations)} expiration dates. First 5: {expirations[:5]}")

# Fetch the Chain for ALL expirations (This might take 30-60 seconds)
options_data = []
print("Fetching options chains")

for date in expirations:
    try:
        # Get the chain for specific date
        opt = ticker.option_chain(date)
        calls = opt.calls
        
        # Calculate days until expiration
        exp_date = datetime.strptime(date, "%Y-%m-%d")
        days_to_exp = (exp_date - datetime.now()).days
        
        # Filter for reasonable data (Liquid options only)
        calls = calls[calls['volume'] > 0].copy()
        
        # Add metadata
        calls['expirationDate'] = date
        calls['daysToExpiration'] = days_to_exp
        
        options_data.append(calls)
    except Exception as e:
        print(f"Skipping {date}: {e}")

df_options = pd.concat(options_data)

# Clean Up
df_options['mid_price'] = (df_options['bid'] + df_options['ask']) / 2
df_options['price'] = np.where(df_options['mid_price'] > 0, df_options['mid_price'], df_options['lastPrice'])

print(f"Data Fetch Complete. Loaded {len(df_options)} call options.")
print(df_options[['strike', 'price', 'daysToExpiration', 'impliedVolatility']].head())

Found 29 expiration dates. First 5: ('2026-01-30', '2026-02-02', '2026-02-03', '2026-02-04', '2026-02-05')
Fetching options chains
Data Fetch Complete. Loaded 3897 call options.
   strike    price  daysToExpiration  impliedVolatility
0   415.0  276.650                -1           4.982426
1   420.0  271.565                -1           4.827152
2   425.0  266.585                -1           4.738285
3   430.0  261.780                -1           4.748051
4   435.0  256.740                -1           4.625004


In [2]:
import scipy.stats as si

# Global Inputs (Spot Price & Risk-Free Rate)
try:
    r = yf.Ticker("^TNX").history(period="1d")['Close'].iloc[-1] / 100
except:
    r = 0.045 # Fallback to 4.5%

# Get current SPY price ('S')
S = ticker.history(period="1d")['Close'].iloc[-1]

print(f"Global Inputs -> Spot Price (S): ${S:.2f}, Risk-Free Rate (r): {r:.2%}")

# Helper Functions for Black-Scholes and Implied Volatility

def norm_pdf(x):
    """Standard Normal Probability Density Function"""
    return (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x**2)

def norm_cdf(x):
    """Standard Normal Cumulative Distribution Function"""
    return si.norm.cdf(x, 0.0, 1.0)

def calculate_d1_d2(S, K, T, r, sigma):
    """Calculates d1 and d2 for Black-Scholes"""
    if sigma <= 0: return 0, 0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

def black_scholes_call(S, K, T, r, sigma):
    """Calculate theoretical Call Price"""
    d1, d2 = calculate_d1_d2(S, K, T, r, sigma)
    return S * norm_cdf(d1) - K * np.exp(-r * T) * norm_cdf(d2)

def calculate_vega(S, K, T, r, sigma):
    """Calculate Vega"""
    d1, _ = calculate_d1_d2(S, K, T, r, sigma)
    return S * norm_pdf(d1) * np.sqrt(T)

# 3. Newton-Raphson Solver
def get_implied_volatility(market_price, S, K, T, r):
    """
    Solves for sigma using Newton-Raphson.
    """
    # Max iterations and precision tolerance
    MAX_ITER = 100
    PRECISION = 1e-5
    
    # Initial Guess
    sigma = 0.5
    
    for i in range(MAX_ITER):
        # Calculate Model Price and Vega with current sigma
        price = black_scholes_call(S, K, T, r, sigma)
        vega = calculate_vega(S, K, T, r, sigma)
        diff = market_price - price
        
        # Check for convergence
        if abs(diff) < PRECISION:
            return sigma
        if abs(vega) < 1e-8:
            return np.nan
        
        # Update sigma
        sigma = sigma + diff / vega
    return np.nan

print("Functions defined.")

Global Inputs -> Spot Price (S): $690.26, Risk-Free Rate (r): 4.25%
Functions defined.


In [3]:
# Convert Days to Years (T)
df_options['T'] = df_options['daysToExpiration'] / 365.0
df_options = df_options[df_options['T'] > 0.001].copy()

print("Calculating Implied Volatility")

# Solver
df_options['Calculated_IV'] = df_options.apply(
    lambda row: get_implied_volatility(
        row['price'],
        S,
        row['strike'],
        row['T'],
        r
    ), axis=1
)
# Remove failures (NaN) and unrealistic outliers (IV > 300% or IV < 1%)
df_clean = df_options.dropna(subset=['Calculated_IV']).copy()
df_clean = df_clean[(df_clean['Calculated_IV'] > 0.01) & (df_clean['Calculated_IV'] < 3.0)]

print(f"Calculation Complete. Successfully solved {len(df_clean)} contracts.")
print(df_clean[['strike', 'expirationDate', 'price', 'Calculated_IV']].head())

Calculating Implied Volatility
Calculation Complete. Successfully solved 3283 contracts.
   strike expirationDate   price  Calculated_IV
0   615.0     2026-02-02  76.825       1.033777
1   630.0     2026-02-02  61.695       0.841650
2   645.0     2026-02-02  46.685       0.668088
3   650.0     2026-02-02  41.780       0.621451
4   655.0     2026-02-02  36.960       0.581601


In [4]:
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install nbformat --upgrade

Note: you may need to restart the kernel to use updated packages.


In [9]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# ==========================================
# FINAL "CINEMATIC" RIBBON BUILDER
# ==========================================

# --- 1. Filter Data (THE FIX) ---
# We cut off anything > 0.8 years (approx 9 months). 
# This removes the "Boring Blue Sea" and zooms in on the "Smile Action."
mask = (df_clean['strike'] > S * 0.80) & (df_clean['strike'] < S * 1.20) & \
       (df_clean['T'] > 0.04) & (df_clean['T'] < 0.8) # <--- KEY CHANGE

df_slice = df_clean[mask].copy()
df_slice = df_slice.sort_values(by=['T', 'strike'])

# --- 2. Build Polynomial Ribbons ---
dense_strikes = np.linspace(df_slice['strike'].min(), df_slice['strike'].max(), 50)
final_X = []
final_Y = []
final_Z = []

unique_times = sorted(df_slice['T'].unique())

print(f"Modeling {len(unique_times)} ribbons (Short-Term Focus)...")

for t in unique_times:
    expiry_data = df_slice[df_slice['T'] == t]
    if len(expiry_data) < 5: continue
        
    current_strikes = expiry_data['strike'].values
    current_ivs = expiry_data['Calculated_IV'].values
    
    try:
        # Degree 3 Polynomial (Smooth Smile)
        coeffs = np.polyfit(current_strikes, current_ivs, 3)
        poly_func = np.poly1d(coeffs)
        smooth_iv = poly_func(dense_strikes)
        smooth_iv = np.maximum(smooth_iv, 0.05)
        
        final_X.append(dense_strikes)
        final_Y.append(np.full(50, t))
        final_Z.append(smooth_iv)
    except:
        continue

X = np.array(final_X)
Y = np.array(final_Y)
Z = np.array(final_Z)

# --- 3. Dynamic Coloring ---
vol_min = np.percentile(Z, 1)
vol_max = np.percentile(Z, 99)

# --- 4. Plot ---
fig = go.Figure(data=[go.Surface(
    x=X, y=Y, z=Z,
    colorscale='Plasma', # Switch to 'Plasma' (Better contrast for Blue/Purple areas)
    cmin=vol_min, cmax=vol_max,
    opacity=0.95,
    lighting=dict(ambient=0.4, diffuse=0.9, fresnel=0.5, specular=1.0, roughness=0.1),
    lightposition=dict(x=0, y=0, z=10000),
    contours_z=dict(show=True, usecolormap=True, highlightcolor="white", project_z=True, width=2)
)])

fig.update_layout(
    title={
        'text': f"S&P 500 VOLATILITY SURFACE (SHORT TERM): {pd.Timestamp.now().strftime('%H:%M:%S')} EST",
        'y': 0.9, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top',
        'font': {'size': 20, 'color': 'white'}
    },
    scene=dict(
        xaxis=dict(title=dict(text='Strike ($)', font=dict(color='silver')), backgroundcolor="rgb(10, 10, 10)", gridcolor="#333", showbackground=True, tickfont=dict(color='silver')),
        yaxis=dict(title=dict(text='Time to Expiry (Yrs)', font=dict(color='silver')), backgroundcolor="rgb(10, 10, 10)", gridcolor="#333", showbackground=True, tickfont=dict(color='silver')),
        zaxis=dict(
            title=dict(text='Implied Volatility', font=dict(color='silver')), 
            backgroundcolor="rgb(10, 10, 10)", 
            gridcolor="#333", 
            showbackground=True, 
            range=[0, vol_max * 1.1], 
            tickfont=dict(color='silver')
        ),
        aspectratio=dict(x=1, y=1, z=0.5),
        camera=dict(eye=dict(x=1.8, y=1.8, z=0.8)) # Zoomed out slightly to see the shape
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    paper_bgcolor='black',
    template='plotly_dark',
    width=1200, height=800
)

fig.show()

Modeling 13 ribbons (Short-Term Focus)...
